In [639]:
from docxtpl import DocxTemplate
import pandas as pd
import os
import datetime as dt
import locale
locale.setlocale(locale.LC_TIME,'')

'French_France.1252'

### Algo de génération des fichiers word

In [640]:

def word_generate(doc_folder, excel_folder, file_field, dest_folder, col_file_name, var1=None, var2=None):
    # doc_folder correspond au répertoire contenant les fichiers word à alimenter
    # file_field correspond à la colonne contenant les noms des fichiers à exporter
    # dest_folder correspond au répertoire de destination des fichiers à exporter
    # col_file_name correspond à la liste des colonnes utilisées pour nommer les fichiers    
    # Etape 1 chargement du fichier excel alimentant les fichiers word
    # var1 Nom du champ exploité pour ranger les résultats, création de répertoire
    # var2 Nom du champ exploité pour ranger les résultats, création de répertoire

    # Etape 1 chargement du fichier excel contenant les valeurs destinées à alimenter les templates word
    augm=pd.read_excel(excel_folder)    
    list_context = augm.columns
    
    # Etape 2, gestion des dates dans le fichier excel
    for col in augm.columns:
        if str(augm[col].dtypes)=="datetime64[ns]":
            augm[col]= augm[col].dt.date
            for i, val in enumerate(augm.iterrows()):
                date_str=str(augm.at[i,col])
                if str(augm.at[i, col]) != "NaT":
                    augm.at[i, col] = dt.datetime.strptime(date_str, '%Y-%m-%d').date()
                    augm.at[i, col] = augm.at[i, col].strftime("%d %B %Y")
            augm[col] = augm[col].astype(str)
    
    # Etape 3 création du contexte
    for i, val in enumerate(augm.iterrows()):
        context={}
        for col in list_context:
            context[col] = augm.at[i, col]
        try:
            folder_world = doc_folder + "\\" + augm.at[i, file_field]
        except:
            print("fichier introuvable")
         
    # Etape 4 Création du nom d'export
        name_file = ""
        for col in col_file_name:
            if name_file =="":
                name_file= augm.at[i,col]
            else:
                name_file = name_file + " " + augm.at[i,col]
        name_file = name_file + ".docx"
        
    # Etape 5 Gestion des répertoires d'export dans le cas d'une ventillation des exports suivant des champs
        if var1 =="" and var2=="":   # Option 1: pas de variable
            dest = export_folder + r"\\" + name_file

        if var1 != "" and var2 == "": # Option 2: 1 variable
            dest = export_folder + r"\\" + augm.at[i,var1] + "\\" 
            if os.path.exists(dest2)== False:
                try:
                    os.mkdir(dest) 
                    dest= dest + "\\" + name_file
                except:
                    print("Creation of the directory %s failed" % dest)

        if var1 != "" and var2 != "": # Option 3: 2 variables
            dest1 = export_folder + "\\" + augm.at[i,var1]
            dest2 = dest1 + "\\" + augm.at[i,var2]
            if os.path.exists(dest2)== False:
                try:
                    os.mkdir(dest1)
                    os.mkdir(dest2)                
                except OSError:
                    print("Creation of the directory %s failed" % dest)
            dest = dest2 + "\\" + name_file
        
        # Etape 6: Génération des words
        doc= DocxTemplate(folder_world)
        doc.render(context)
        doc.save(dest)    

In [641]:
augm =r"C:\Users\sgasmi\Desktop\Dev\Com&Ben\Notification des Augmentations\Bordereau pour Notifications_IBF fichier test.xlsx"
folder_word = r"C:\Users\sgasmi\Desktop\Dev\Com&Ben\Notification des Augmentations\Template"
export_folder = r"C:\Users\sgasmi\Desktop\Dev\Com&Ben\Export"



word_generate(doc_folder=folder_word,excel_folder=augm, file_field="Fichiers_export",col_file_name=["Nom","Prénom","MYHR_ID"], dest_folder=export_folder
             ,var1= "Nom_Prénom_RRH",var2 = "Nom_Prénom_Manager")